<a href="https://colab.research.google.com/github/hurchey/NCAA-Basketball-Machine-Learning-Prediction/blob/main/March_Madness_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Setting up the data sets

Download Files
https://www.kaggle.com/competitions/march-machine-learning-mania-2023/data

basically we are trying to predict the march madness winner for March 2024

In [ ]:
import os
import numpy as np
import pandas as pd
import xgboost as xgb # ask bojan tunguz why you have to use it
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import brier_score_loss # competition metric
from sklearn.metrics import mean_absolute_error # win margin proxy model metric
from scipy.interpolate import UnivariateSpline # interpolate win margin model to probabilities

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This is downloading all of the needed files

In [ ]:
tourney_results = pd.read_csv('/content/drive/MyDrive/March_Madness_2023/MNCAATourneyDetailedResults.csv')

seeds = pd.read_csv('/content/drive/MyDrive/March_Madness_2023/MNCAATourneySeeds.csv')

regular_results = pd.read_csv('/content/drive/MyDrive/March_Madness_2023/MRegularSeasonDetailedResults.csv')

fivethirtyeight = pd.read_csv('/content/drive/MyDrive/March_Madness_2023/fivethirtyeight_ncaa_forecasts.csv')

# Step 2: Setting up each data representation for LSTM Model

Having all of the team id associated to each team name

In [ ]:
import pandas as pd

# The file has columns named 'TeamID' and 'TeamName'
team_id_to_name = pd.Series(fivethirtyeight.team_name.values, index=fivethirtyeight.team_id).to_dict()

print(team_id_to_name)

{41: 'Connecticut', 21: 'San Diego State', 2226: 'Florida Atlantic', 2390: 'Miami (FL)', 156: 'Creighton', 2306: 'Kansas State', 2250: 'Gonzaga', 251: 'Texas', 333: 'Alabama', 163: 'Princeton', 2633: 'Tennessee', 127: 'Michigan State', 8: 'Arkansas', 26: 'UCLA', 248: 'Houston', 2752: 'Xavier', 120: 'Maryland', 231: 'Furman', 239: 'Baylor', 142: 'Missouri', 161: 'Fairleigh Dickinson', 150: 'Duke', 96: 'Kentucky', 269: 'Marquette', 2305: 'Kansas', 2608: "Saint Mary's (CA)", 2628: 'Texas Christian', 77: 'Northwestern', 2: 'Auburn', 84: 'Indiana', 221: 'Pittsburgh', 213: 'Penn State', 357: 'Texas A&M-Corpus Christi', 277: 'West Virginia', 232: 'College of Charleston', 258: 'Virginia', 152: 'North Carolina State', 2540: 'UC-Santa Barbara', 328: 'Utah State', 12: 'Arizona', 2509: 'Purdue', 235: 'Memphis', 198: 'Oral Roberts', 309: 'Louisiana-Lafayette', 2507: 'Providence', 147: 'Montana State', 30: 'Southern California', 261: 'Vermont', 47: 'Howard', 356: 'Illinois', 2670: 'Virginia Commonwe

This represents each team's average scoring and fg%

In [ ]:
# Feature Engineering: Calculate team ratings, overall points, and field goal percentages

# Function to calculate field goal percentage
def calculate_fg_percentage(df):
    df['FG%'] = df['WFGM'] / df['WFGA']
    df['LFG%'] = df['LFGM'] / df['LFGA']
    return df

# Apply the function to the tournament and regular season datasets
tourney_results = calculate_fg_percentage(tourney_results)
regular_season_detailed_results = calculate_fg_percentage(regular_results)

# Aggregate statistics for each team and season
# For simplicity, we'll use mean statistics here, but more complex methods could be used

# Winning team stats
winning_stats = regular_season_detailed_results.groupby(['Season', 'WTeamID']).mean().add_prefix('W')
losing_stats = regular_season_detailed_results.groupby(['Season', 'LTeamID']).mean().add_prefix('L')

# Merge winning and losing stats and replace NaN with 0
team_stats = pd.merge(winning_stats, losing_stats, left_index=True, right_index=True, how='outer').fillna(0)

# Calculate combined stats
team_stats['Points'] = team_stats['WWScore'] + team_stats['LLScore']
team_stats['FG%'] = (team_stats['WFG%'] + team_stats['LFG%']) / 2

# Select relevant columns
team_stats = team_stats[['Points', 'FG%']]

# Display the first few rows of the aggregated team stats
team_stats.head(9999999)


<ipython-input-5-3b19ffde851d>:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  winning_stats = regular_season_detailed_results.groupby(['Season', 'WTeamID']).mean().add_prefix('W')
<ipython-input-5-3b19ffde851d>:18: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  losing_stats = regular_season_detailed_results.groupby(['Season', 'LTeamID']).mean().add_prefix('L')


Points       FG%
Season WTeamID LTeamID                      
2003   1102    1102     117.375000  0.533013
               1103     139.178571  0.543878
               1104     129.659091  0.523604
               1105     137.697368  0.523199
               1106     128.283333  0.514452
...                            ...       ...
2023   1477    1473     133.666667  0.459479
               1474     144.000000  0.468801
               1475     144.125000  0.464042
               1476     134.588235  0.452418
               1477     141.850000  0.473902

[2504533 rows x 2 columns]

In [ ]:
# Reducing dataset size by focusing on the most recent seasons (e.g., last 5 seasons)
recent_seasons = regular_season_detailed_results['Season'].max() - 4
recent_season_data = regular_season_detailed_results[regular_season_detailed_results['Season'] >= recent_seasons]

# Simplified feature engineering for the reduced dataset
def calculate_basic_stats(df):
    # Calculate average points scored and field goal percentage for each team and season
    stats = df.groupby(['Season', 'WTeamID']).agg({'WScore': 'mean', 'FG%': 'mean'}).rename(columns={'WScore': 'AvgScore', 'FG%': 'AvgFG%'})
    return stats

# Calculate stats for the recent season data
recent_season_stats = calculate_basic_stats(recent_season_data)

# Display the first few rows of the recent season stats
recent_season_stats.head()


AvgScore    AvgFG%
Season WTeamID                     
2019   1101     73.521739  0.469469
       1102     77.461538  0.492202
       1103     73.800000  0.455895
       1104     77.222222  0.462999
       1105     69.800000  0.456330

This is just more advanced data of the last 5 seasons

In [ ]:
# Filtering the dataset for the most recent 5 seasons
latest_season = regular_season_detailed_results['Season'].max()
recent_season_data = regular_season_detailed_results[regular_season_detailed_results['Season'] >= latest_season - 4]

# Display the first few rows of the recent season data
recent_season_data.head(9999999)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,FG%,LFG%
82041,2019,1,1104,82,1380,62,H,0,27,55,...,30,11,20,8,15,7,2,28,0.490909,0.381818
82042,2019,1,1113,102,1168,94,H,0,33,78,...,26,7,35,7,17,7,3,36,0.423077,0.459459
82043,2019,1,1119,73,1265,69,H,0,22,51,...,24,9,27,8,12,4,0,23,0.431373,0.519231
82044,2019,1,1120,101,1375,58,H,0,38,74,...,22,8,19,9,22,4,5,17,0.513514,0.369565
82045,2019,1,1123,86,1232,69,H,0,32,65,...,18,18,20,11,11,6,5,20,0.492308,0.388060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107629,2023,132,1104,82,1401,63,N,0,27,66,...,24,17,20,6,13,5,2,21,0.409091,0.296875
107630,2023,132,1272,75,1222,65,A,0,27,53,...,28,22,20,11,10,5,4,19,0.509434,0.312500
107631,2023,132,1343,74,1463,65,H,0,23,57,...,19,6,25,12,10,3,7,20,0.403509,0.440000
107632,2023,132,1345,67,1336,65,N,0,24,62,...,15,4,27,14,10,4,2,20,0.387097,0.431373


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Preparing the data for model training
# For simplicity, we'll use average scores and field goal percentages as features
features = recent_season_stats[['AvgScore', 'AvgFG%']]
target = recent_season_stats['AvgScore']  # Predicting the average score as a target

# Training a Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(features, target)

# Making predictions for the most recent season as a proxy for 2024
predictions = rf_model.predict(features)

# Attaching predictions back to the DataFrame
recent_season_stats['PredictedScore'] = predictions

# Sorting teams based on predicted score for the next season (2024 proxy)
predicted_scores_2024 = recent_season_stats.groupby(level=1)['PredictedScore'].mean().sort_values(ascending=False)

# Displaying the top teams based on the model's predictions
top_teams_2024 = predicted_scores_2024.head(10)
top_teams_2024


WTeamID
1211    89.938263
1223    86.961578
1234    85.160513
1331    84.560588
1138    83.836946
1440    83.811404
1267    83.354781
1385    83.268803
1473    83.247830
1104    83.246543
Name: PredictedScore, dtype: float64

This shows each team's average rating and fg% associated with that rating

In [ ]:
# Identifying top-performing teams based on recent seasons' data
# We'll look at the teams with the highest average scores and field goal percentages

# Calculating average scores and field goal percentages for each team across recent seasons
avg_scores = recent_season_stats.groupby(level=1)['AvgScore'].mean()
avg_fg_percentages = recent_season_stats.groupby(level=1)['AvgFG%'].mean()

# Combining these metrics into a single DataFrame
combined_performance = pd.DataFrame({
    'AvgScore': avg_scores,
    'AvgFG%': avg_fg_percentages
})

# Sorting teams based on average scores and field goal percentages
top_teams_by_score = combined_performance.sort_values(by='AvgScore', ascending=False).head(10)
top_teams_by_fg_percentage = combined_performance.sort_values(by='AvgFG%', ascending=False).head(10)

# Displaying the top teams based on average scores and field goal percentages
top_teams_by_score, top_teams_by_fg_percentage


(          AvgScore    AvgFG%
 WTeamID                     
 1211     90.151703  0.537357
 1223     86.982857  0.495889
 1234     85.164351  0.484693
 1331     84.564744  0.495074
 1138     83.840030  0.471581
 1440     83.818077  0.475491
 1267     83.354237  0.482313
 1385     83.264192  0.477265
 1473     83.250000  0.502484
 1104     83.248256  0.462796,
           AvgScore    AvgFG%
 WTeamID                     
 1211     90.151703  0.537357
 1152     76.892857  0.527943
 1355     82.658378  0.517225
 1260     73.007241  0.516290
 1173     75.130021  0.514426
 1161     79.329626  0.514114
 1471     80.196970  0.508242
 1270     81.601991  0.506134
 1225     77.833333  0.505408
 1125     82.866333  0.503968)

# Step 3: Setting up the model for training and testing

This next part is the creation of the model and training that model (LSTM)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# Let's assume 'recent_season_stats' is your DataFrame from earlier steps
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(recent_season_stats)

# Creating sequences
# Define sequence length (number of seasons to look back)
sequence_length = 5

# The generator will create sequences of data points (X) and targets (y)
generator = TimeseriesGenerator(scaled_data, scaled_data, length=sequence_length, batch_size=1)

# Preparing the data for LSTM
X_train, y_train = [], []
for i in range(len(generator)):
    x, y = generator[i]
    X_train.append(x[0])
    y_train.append(y[0])

X_train = np.array(X_train)
y_train = np.array(y_train)

# X_train and y_train are now ready to be used in an LSTM model


In [ ]:
# These should be numpy arrays in the shape (num_samples, num_time_steps, num_features)

import tensorflow.keras.models
import tensorflow.keras.layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

X_train = np.random.rand(1000, 5, 10)  # Example data
y_train = np.random.rand(1000, 1)      # Example target

model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32)


Epoch 1/50
32/32 [==============================] - 4s 9ms/step - loss: 0.1113
Epoch 2/50
32/32 [==============================] - 0s 9ms/step - loss: 0.0845
Epoch 3/50
32/32 [==============================] - 0s 8ms/step - loss: 0.0846
Epoch 4/50
32/32 [==============================] - 0s 11ms/step - loss: 0.0829
Epoch 5/50
32/32 [==============================] - 0s 13ms/step - loss: 0.0826
Epoch 6/50
32/32 [==============================] - 0s 12ms/step - loss: 0.0845
Epoch 7/50
32/32 [==============================] - 0s 12ms/step - loss: 0.0835
Epoch 8/50
32/32 [==============================] - 0s 13ms/step - loss: 0.0836
Epoch 9/50
32/32 [==============================] - 0s 13ms/step - loss: 0.0831
Epoch 10/50
32/32 [==============================] - 0s 13ms/step - loss: 0.0823
Epoch 11/50
32/32 [==============================] - 0s 12ms/step - loss: 0.0822
Epoch 12/50
32/32 [==============================] - 0s 11ms/step - loss: 0.0824
Epoch 13/50
32/32 [=====================

# Step 4: Using the model to predict the 2024 winner

This part is showing what the model thinks will be the top team to win the 2024 Championship

In [ ]:
# Assuming X_2023 is prepared for your 2023 season data in the correct LSTM format
X_2023 = np.random.rand(30, 5, 10)  # Example 2023 data
predictions_2024 = model.predict(X_2023)

# Interpret these predictions to determine the top team
top_team_index = np.argmax(predictions_2024)
top_team_id = fivethirtyeight.loc[top_team_index, 'team_id']
top_team_name = team_id_to_name[top_team_id]
print(f"Predicted top team for 2024: {top_team_name}")

1/1 [==============================] - 2s 2s/step
Predicted top team for 2024: Auburn
